In [28]:
from interactions import get_interactions
import numpy as np
import os

In [29]:
X = 128
Y = 128
Y2 = int(Y/2)

Get interactions for first sublattice for a specific disorder sample

In [30]:
interactions_new = np.loadtxt('results/HamiltonianTest/periodic_boundary/bonds_e0.txt')

In [31]:
interactions = get_interactions(interactions_new[:,256:], X, X)

Get black and white first sublattice of first update step for current error sample 

In [32]:
# Read the content of the text file
with open('results/HamiltonianTest/periodic_boundary/lattice_256x128_e0_T3.000000_it00000002_.txt') as file:
    content = file.readlines()

# Remove any newline characters
content = [line.strip() for line in content]

# Convert the content into a numpy array of integers
lattice = np.array([[int(char) for char in line] for line in content])

lattice[lattice==0] = -1

In [33]:
lattice = lattice[:,128:]

In [34]:
lattice_b = np.zeros((X,Y2))
lattice_w = np.zeros((X,Y2))
# Get black and white lattices
for i in range(X):
    if i%2 == 0:
        lattice_b[i,:] = lattice[i,0::2]
        lattice_w[i,:] = lattice[i,1::2]
    else:
        lattice_b[i,:] = lattice[i,1::2]
        lattice_w[i,:] = lattice[i,0::2]

In [35]:
hamiltonian = 0
for is_black in (True, False):
    for tid in range(X*Y2):

        i = int(tid/Y2)
        j = tid % Y2

        ipp = (i + 1) if (i + 1 < X) else 0
        inn = (i - 1) if (i - 1 >= 0) else X - 1
        jpp = (j + 1) if (j + 1 < Y2) else 0
        jnn = (j - 1) if (j - 1 >= 0) else Y2 - 1

        c_up = 1-int(inn/(X-1))
        c_down = 1-int((i+1)/X)

        if (is_black):

            icpp = 2*(X-1)*Y2 + 2*(Y2*(i+1) + j) + i%2
            
            icnn = 2*(X-1)*Y2 + 2*(Y2*(inn+1) + j) + i%2

            joff = jpp if (i%2) else jnn
            
            if (i % 2):
                if (j+1 >= Y2):
                    jcoff = 2*(i*Y2 + j + 1) - 1
                
                else:
                    jcoff = 2*(i*Y2 + joff) - 1
            else:
                jcoff = 2 * (i*Y2 + joff) + 1

            hamiltonian += lattice_b[i,j]*(lattice_w[inn,j]*interactions[icnn] + lattice_w[ipp,j]*interactions[icpp] + lattice_w[i,j]*interactions[2*(i*Y2+j)] + lattice_w[i, joff]*interactions[jcoff])

        else:

            icpp = 2*(X - 1)*Y2 + 2*(Y2*(i+1) + j) + (i+1)%2
            icnn = 2*(X - 1)*Y2 + 2*(Y2*(inn+1) + j) + (i+1)%2

            joff = jnn if (i%2) else jpp

            if (i%2):
                jcoff = 2*(i*Y2 + joff) + 1
            else:
                if (j+1 >= Y2):
                    jcoff = 2*(i*Y2 + j + 1) - 1
                else:
                    jcoff = 2*(i*Y2 + joff) - 1

            hamiltonian += lattice_w[i,j]*(lattice_b[inn,j]*interactions[icnn] + lattice_b[ipp,j]*interactions[icpp] + lattice_b[i,j]*interactions[2*(i*Y2+j)] + lattice_b[i, joff]*interactions[jcoff])

In [36]:
hamiltonian

17140.0